In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd "drive/MyDrive/TUM/TUM XAI/generators/Galactica/output"

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1m6AYZbd3XOUMFqsgOLrRp8rl9q4ZPFfZ/TUM XAI/Galactica_NEW


In [ ]:
%cd output

In [ ]:
!ls

combine_output.ipynb	 galactica_3000_3500.csv
galactica_0_500.csv	 galactica_3500_3650.csv
galactica_1000_1250.csv  galactica_3650_3800.csv
galactica_1250_1500.csv  galactica_3800_4000.csv
galactica_1500_1750.csv  galactica_4000_4500.csv
galactica_1750_2000.csv  galactica_4500_5000.csv
galactica_2000_2450.csv  galactica_500_1000.csv
galactica_2450_2500.csv  galactica_output_complete_cleaned.csv
galactica_2500_3000.csv


In [ ]:
import os
import re
files = os.listdir()
p = re.compile('galactica_\d+_\d+.csv')
files = [ s for s in files if p.match(s) ]
files.sort()
files

['galactica_0_500.csv',
 'galactica_1000_1250.csv',
 'galactica_1250_1500.csv',
 'galactica_1500_1750.csv',
 'galactica_1750_2000.csv',
 'galactica_2000_2450.csv',
 'galactica_2450_2500.csv',
 'galactica_2500_3000.csv',
 'galactica_3000_3500.csv',
 'galactica_3500_3650.csv',
 'galactica_3650_3800.csv',
 'galactica_3800_4000.csv',
 'galactica_4000_4500.csv',
 'galactica_4500_5000.csv',
 'galactica_500_1000.csv']

In [ ]:
import pandas as pd
df_li = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_li.append(df)

In [ ]:
all_files_df = pd.concat(df_li, axis=0, ignore_index=True)[["title", "abstract", "introduction", "conclusion"]]
all_files_df

,title,abstract,introduction,conclusion
0,Fermi-liquid effects in the transresistivity i...,Low-temperature transresistivities for double-...,The formation of composite fermions (CFs) at f...,1. The transresistivities show an abrupt incre...
1,Near-Infrared Spectra of the Black Hole X-Ray ...,"Spectra of the black hole X-ray binary, A0620-...","The black hole X-ray binary, A0 620-00, is one...",1. The near-infrared emission of A0620-00 is d...
2,Ising-like dynamics and frozen states in syste...,In this article it is shown that the Ising-lik...,"In recent years, magnetic properties of ultraf...",In this contribution we have considered the fo...
3,Fluctuations in glassy systems,We discuss the theory of fluctuations in glass...,The physics of fluctuations in disordered syst...,1. A theory of fluctuations in glasses has bee...
4,Three dimensional cooling and trapping with a ...,We report on the generation and detection of ...,The optical cooling of atoms to temperatures o...,The results suggest that the atoms may be in a...
...,...,...,...,...
4995,Holographic Dual of Linear Dilaton Black Hole ...,"In this letter, we attempt to explore the holo...","Holographic duality, which is a powerful tool ...",We have studied the holographic dual of the bl...
4996,The Coherent-Constructible Correspondence and ...,"We describe the natural correspondence, in the...",The paper is based on the work of D. Arinkin a...,The paper is a generalization of the paper [2]...
4997,Independent Attacks in Imperfect Settings; A C...,This paper analyzes the security of the recent...,1. The basic idea of the QKD scheme proposed i...,1. The above QKD scheme is secure against an i...
4998,Polaron contributions to the biexciton binding...,The temperature behavior of biexciton binding ...,Self-assembled quantum dots (QDs) in semicondu...,The temperature dependence of the ${E}^{\mathr...


# preprocessing

In [ ]:
all_files_df["conclusion"] = all_files_df["conclusion"].fillna("")

In [ ]:
def clean_special_tokens(row):
  tags = [
    "[START_REF]",
    "[END_REF]",
    "[START_SUP]",
    "[END_SUP]",
    "[START_DNA]",
    "[END_DNA]",
    "[START_AMINO]",
    "[END_AMINO]",
    "[START_SMILES]",
    "[END_SMILES]",
    "[START_I_SMILES]",
    "[END_I_SMILES]",
  ]
  reference_index = 1
  reference = {}
  title = row["title"]
  abstract = row["abstract"]
  introduction = row["introduction"]
  conclusion = row["conclusion"]

  while abstract.find("[START_REF]") >= 0:
    start = abstract.find("[START_REF]")
    end = abstract.find("[END_REF]") + len("[END_REF]")
    ref = abstract[start:end]

    if ref not in reference:
        reference[ref] = reference_index
        abstract = abstract.replace(ref, str(reference_index))
        reference_index += 1
    else:
        abstract = abstract.replace(ref, str(reference[ref]))

  for tag in tags:
    abstract = abstract.replace(tag, "")

  while introduction.find("[START_REF]") >= 0:
    start = introduction.find("[START_REF]")
    end = introduction.find("[END_REF]") + len("[END_REF]")
    ref = introduction[start:end]

    if ref not in reference:
        reference[ref] = reference_index
        introduction = introduction.replace(ref, str(reference_index))
        reference_index += 1
    else:
        introduction = introduction.replace(ref, str(reference[ref]))

  for tag in tags:
    introduction = introduction.replace(tag, "")
  # conclusion
  while conclusion.find("[START_REF]") >= 0:

      start = conclusion.find("[START_REF]")
      end = conclusion.find("[END_REF]") + len("[END_REF]")
      if start > end:
          conclusion = conclusion[:start]
          break
      ref = conclusion[start:end]

      if ref not in reference:
          reference[ref] = reference_index
          conclusion = conclusion.replace(ref, str(reference_index))
          reference_index += 1
      else:
          conclusion = conclusion.replace(ref, str(reference[ref]))

  for tag in tags:
    conclusion = conclusion.replace(tag, "")

  return pd.Series([title, abstract, introduction, conclusion], index=['title', 'abstract', 'introduction', 'conclusion'])
  # return {"abstract":abstract, "introduction":introduction, "conclusion":conclusion}

In [ ]:
all_files_df = all_files_df.apply(clean_special_tokens, axis = 1)

In [ ]:
def remove_from_section(df, remove_after_term, section = "conclusion"):
  # remove a regular expression from a section.
  # faster extract.
  # consider "title" or "# title" or "# 1 title" or "# 1. title".
  df_temp = df[df[section].str.contains(remove_after_term)][section].str.extract(f'([\s\S]*?)(?:# (?:\d*\.? )?)?{remove_after_term}')[0]
  display(pd.DataFrame(df_temp))
  df.loc[df[section].str.contains(remove_after_term), section] = df_temp
  return df

In [ ]:
all_files_df[all_files_df["abstract"].str.contains('ABSTRACT')][["abstract"]]

,abstract
92,ABSTRACT The effect of non-Gaussian curvature ...
907,"ABSTRACT In this paper, we give a coding schem..."
1056,ABSTRACT At least five species of aquatic bird...
1117,ABSTRACT We give expressions for the cross cap...
1208,ABSTRACT This paper presents a fast optimizati...
1293,ABSTRACT A spatial graph is an intersection gr...
1472,ABSTRACT Let f be an integral polynomial over ...
1696,"ABSTRACT In this paper, we derive an explicit ..."
1887,ABSTRACT The present paper describes a new co...
1955,"ABSTRACT Multifractal processes, introduced by..."


In [ ]:
all_files_df[all_files_df["abstract"].str[0:8] == "ABSTRACT"]

,title,abstract,introduction,conclusion
92,The effect of non-Gaussian curvature perturbat...,ABSTRACT The effect of non-Gaussian curvature ...,Primordial black holes (PBHs) are formed from ...,The effect of non-Gaussianity on the formation...
907,Network Coding Capacity Regions via Entropy Fu...,"ABSTRACT In this paper, we give a coding schem...",Network coding is a new technique for achievin...,- The proposed coding scheme is optimal for th...
1056,American coot collective on-water dynamics,ABSTRACT At least five species of aquatic bird...,American coot (Fulica americanaina) population...,We demonstrate a novel approach to describe th...
1117,Intrinsic Invariants of Cross Caps,ABSTRACT We give expressions for the cross cap...,The cross caps in projective n-space are defin...,The cross caps in projective n-spaces are defi...
1208,Efficient MRF Energy Minimization via Adaptive...,ABSTRACT This paper presents a fast optimizati...,The Markov random field (MRF) is a powerful to...,The proposed algorithm can be applied to a wid...
1293,A homotopy classification of two-component spa...,ABSTRACT A spatial graph is an intersection gr...,Spatial graphs are intersection graphs of a pl...,(1) If two spatial graphs and have the same ...
1472,An effective uniform Artin-Rees lemma,ABSTRACT Let f be an integral polynomial over ...,Let f(x) = a0 + a1x + · · · + an x n be an int...,1. We have proved a uniform Artin-Rees lemma f...
1696,An explicit formula for computing Bell numbers...,"ABSTRACT In this paper, we derive an explicit ...","In 1914, G. Bell [1] introduced a sequence of ...",$$B_n=\frac{n!}{2^n}\sum_{k=0}^{n}\frac{(-1)^{...
1887,Slime Mould Logic Gates Based on Frequency Cha...,ABSTRACT The present paper describes a new co...,- Slime moulds are simple and easy to handle ...,- The frequency of the oscillation of the ele...
1955,Some sample path properties of multifractional...,"ABSTRACT Multifractal processes, introduced by...","Multifractal processes, introduced by Abry and...",1. The main results of this paper are new gene...


remove ABSTRACT if shown in the beginning 

In [ ]:
all_files_df.loc[all_files_df["abstract"].str[0:8] == "ABSTRACT", "abstract"] = all_files_df["abstract"].str[9:]

In [ ]:
all_files_df[all_files_df["abstract"].str[0:8] == "ABSTRACT"]

,title,abstract,introduction,conclusion


remove Introductions found in conclusion

In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Introduction:')][["conclusion"]]

,conclusion
14,(a) The Amati relation holds only for the cut ...
46,- 1.3.1. The MOST data - 1.3.2. The photome...
69,The results obtained are in excellent agreemen...
100,(5'' ) in the case when d = n ≥ 3 and X has co...
178,# 1.1.1 The Phenomenology of Gauge Symmetry Br...
...,...
4859,(1) The two-body matrix elements of the relati...
4884,- - - - - - - - [Provenance](#Provenanc...
4919,We present a catalog of 1879 EBs with a period...
4926,"- In this paper, we present a novel analytical..."


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Introduction:", section = "conclusion")

,0
14,(a) The Amati relation holds only for the cut ...
46,- 1.3.1. The MOST data - 1.3.2. The photome...
69,The results obtained are in excellent agreemen...
100,(5'' ) in the case when d = n ≥ 3 and X has co...
178,# 1.1.1 The Phenomenology of Gauge Symmetry Br...
...,...
4859,(1) The two-body matrix elements of the relati...
4884,- - - - - - - - [Provenance](#Provenanc...
4919,We present a catalog of 1879 EBs with a period...
4926,"- In this paper, we present a novel analytical..."


In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Conclusion:')]

,title,abstract,introduction,conclusion
178,Phenomenology from the Landscape of String Vacua,The landscape of string theory predicts a wide...,The landscape of string vacua is a rich source...,# 1.1.1 The Phenomenology of Gauge Symmetry Br...
196,Analyticity and propagation of plurisubharmoni...,"Assume ( X, ω ) is a Hermitian symplectic mani...","1.1. Statement of the main results. Let ( X, ω...",1.2. Applications. Let us briefly discuss the ...
591,A Kinetics Of The Non-Equilibrium Universe. II...,It is shown that the laws of irreversible ther...,- The state of the world at the end of the las...,- The world is in the state of the maximum ent...
607,Dynamical decoupling and noise spectroscopy wi...,The noise properties of a resistively shunted ...,- The interaction of a qubit with its environm...,- We have investigated the coherence of a flux...
659,The Eclipsing Cataclysmic Variables PHL 1445 a...,Photometric and spectroscopic observations of ...,- 1.1.1 PHL 1445 - 1.1.2 GALEX J003535+46235...,- 1.2.1.1 PHL 1455 - 1.2.1.2 GALEX J 010236....
734,Gamma-Ray Constraints on Maximum Cosmogenic Ne...,"In this paper, Gamma-Ray Observation of UHECRs...",The cosmic ray flux is measured by a variety o...,1. Cosmic-ray flux from GRBs (GRB980425): T...
805,Approximate Distance Oracles with Improved Pre...,Distance oracle is defined as a data structur...,Distance oracles are defined as data structure...,# 2.2 Approximate Distance Oracles for Bounded...
1028,The near-critical scaling window for directed ...,We consider a model of oriented self-avoiding ...,The study of random walks on trees has a long ...,"In this paper, we study a model of oriented se..."
1281,Conditional mutual information and self-commut...,We derive an equation which directly connects ...,"Let $X_t$ denote a continuous-time, stationary...",$\displaystyle\lim_{\tau\rightarrow\frac{1}{2}...
1328,Euclidean plane and its relatives; a minimalis...,"Euclidean plane, for its most part, is unders...",Euclidean plane and its relatives; a minimalis...,Appendix: Euclidean plane and its relatives...


remove conclusions showed up in conclusion section after finding a conclusion.

In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Conclusion:", section = "conclusion")

,0
178,# 1.1.1 The Phenomenology of Gauge Symmetry Br...
196,1.2. Applications. Let us briefly discuss the ...
591,- The world is in the state of the maximum ent...
607,- We have investigated the coherence of a flux...
659,- 1.2.1.1 PHL 1455 - 1.2.1.2 GALEX J 010236....
734,1. Cosmic-ray flux from GRBs (GRB980425): T...
805,# 2.2 Approximate Distance Oracles for Bounded...
1028,"In this paper, we study a model of oriented se..."
1281,$\displaystyle\lim_{\tau\rightarrow\frac{1}{2}...
1328,Appendix: Euclidean plane and its relatives...


remove refs

In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Reference:')][["conclusion"]]

,conclusion
24,The heat capacity of PbS crystals was measured...
25,The matrix product ansatz is the starting poin...
108,1. The standard formulation of quantum probabi...
115,- The presence of a strong magnetic field in t...
145,1. The thermal emission from the accretion pro...
...,...
4798,- NZ scientists will be able to contribute to ...
4890,- The proposed CAS is useful for the selection...
4907,\begin{abstract} We prove that the anisot...
4939,-. Reference: -. -. -.


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Reference:", section = "conclusion")

,0
24,The heat capacity of PbS crystals was measured...
25,The matrix product ansatz is the starting poin...
108,1. The standard formulation of quantum probabi...
115,- The presence of a strong magnetic field in t...
145,1. The thermal emission from the accretion pro...
...,...
4798,- NZ scientists will be able to contribute to ...
4890,- The proposed CAS is useful for the selection...
4907,\begin{abstract} We prove that the anisot...
4939,-.


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "References:", section = "conclusion")

,0
63,6. What are the main directions of future rese...
75,
227,The results presented in this work are a step ...
251,
345,
...,...
4855,The attractor mechanism is a kind of distillat...
4862,
4881,
4951,This paper suggests that the N-queens problem ...


remove abstracts in conclusion

In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Abstract:')][["conclusion"]]

,conclusion
466,Axinos as Dark Matter Particles Authors: ...
958,90-degree Ferroelectric Domains in PbTiO 3 : ...
970,"$$|u(x,t) - u_{\varepsilon } (x,t)| = o(\varep..."
988,5. The dynamics of nestedness predicts the evo...
1005,The workshop was hosted by Lawrence Berkeley N...
1056,We demonstrate a novel approach to describe th...
1328,Appendix: Euclidean plane and its relatives...
1415,- The role of magnetic field strength and nume...
1417,"* ""Matrix Inversion is as easy as exponentiati..."
1548,Relativistic deformation in deformed nuclei by...


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Abstract:", section = "conclusion")

,0
466,Axinos as Dark Matter Particles Authors: ...
958,90-degree Ferroelectric Domains in PbTiO 3 : ...
970,"$$|u(x,t) - u_{\varepsilon } (x,t)| = o(\varep..."
988,5. The dynamics of nestedness predicts the evo...
1005,The workshop was hosted by Lawrence Berkeley N...
1056,We demonstrate a novel approach to describe th...
1328,Appendix: Euclidean plane and its relatives...
1415,- The role of magnetic field strength and nume...
1417,"* ""Matrix Inversion is as easy as exponentiati..."
1548,Relativistic deformation in deformed nuclei by...


remove Acknowledgments in conclusion

In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Acknowledgments')][["conclusion"]]

,conclusion
12,We study the two-dimensional +-J spin-glass mo...
17,"In this paper, we explain why the method suffe..."
20,We find that the critical conductance of a cle...
52,- A universal interacting crossover regime is ...
57,"In this work, we derive an analytical expressi..."
...,...
4955,We have shown that the interaction between sup...
4958,(1) The inner region is turbulent and opticall...
4971,We think that this paper provides a good basis...
4973,1. The Regge models with higher number of Regg...


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Acknowledgments", section = "conclusion")

,0
12,We study the two-dimensional +-J spin-glass mo...
17,"In this paper, we explain why the method suffe..."
20,We find that the critical conductance of a cle...
52,- A universal interacting crossover regime is ...
57,"In this work, we derive an analytical expressi..."
...,...
4955,We have shown that the interaction between sup...
4958,(1) The inner region is turbulent and opticall...
4971,We think that this paper provides a good basis...
4973,1. The Regge models with higher number of Regg...


remove Authors in conclusion

In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Authors')][["conclusion"]]

,conclusion
466,Axinos as Dark Matter Particles Authors: ...
1007,The results show that it is possible to delive...
1968,The Hall viscosity of the quantum Hall systems...
2356,- The project.Net-HPC provides a Grid infrastr...
2642,- The theory presented in this paper can be us...
3080,The bistability parameter increases with the i...
3468,"In this paper, we propose a neural network tra..."
3645,This work is a part of our ongoing research on...
4037,This paper presents a novel sequential recomme...


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Authors:", section = "conclusion")

,0
466,Axinos as Dark Matter Particles
1007,The results show that it is possible to delive...
1968,The Hall viscosity of the quantum Hall systems...
2356,- The project.Net-HPC provides a Grid infrastr...
2642,- The theory presented in this paper can be us...
3080,The bistability parameter increases with the i...
3468,"In this paper, we propose a neural network tra..."
3645,This work is a part of our ongoing research on...
4037,This paper presents a novel sequential recomme...


remove Keywords in conclusion

In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Keywords:')][["conclusion"]]

,conclusion
108,1. The standard formulation of quantum probabi...
279,$$\frac{1}{\log n}\sum_{\sigma=1-\delta}^1\sum...
361,The proposed method is suitable for solving la...
372,- The coin degree of freedom of a QW has been ...
386,"- The jet reconstruction, jet properties and i..."
...,...
4882,"- - - Author(s): E. Dziembowski, S...."
4885,1. The Hardy-Poincare' inequality with boundar...
4890,- The proposed CAS is useful for the selection...
4944,1. The dynamic behavior of the urban road netw...


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Keywords:", section = "conclusion")

,0
108,1. The standard formulation of quantum probabi...
279,$$\frac{1}{\log n}\sum_{\sigma=1-\delta}^1\sum...
361,The proposed method is suitable for solving la...
372,- The coin degree of freedom of a QW has been ...
386,"- The jet reconstruction, jet properties and i..."
...,...
4882,"- - - Author(s): E. Dziembowski, S...."
4885,1. The Hardy-Poincare' inequality with boundar...
4890,- The proposed CAS is useful for the selection...
4944,1. The dynamic behavior of the urban road netw...


In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Authors:')][["conclusion"]]

,conclusion


remove Citation in conclusion

In [ ]:
all_files_df[all_files_df["conclusion"].str.contains('Citation:')][["conclusion"]]

,conclusion
1894,- [The VoG framework](http://vog.sourceforge.n...
2356,- The project.Net-HPC provides a Grid infrastr...
3905,- We propose a novel approach to tackle the ch...
4241,- We propose an approach for augmenting the co...
4252,-------------------- The LGFS model is ca...


In [ ]:
all_files_df = remove_from_section(all_files_df, remove_after_term = "Citation:", section = "conclusion")

,0
1894,- [The VoG framework](http://vog.sourceforge.n...
2356,- The project.Net-HPC provides a Grid infrastr...
3905,- We propose a novel approach to tackle the ch...
4241,- We propose an approach for augmenting the co...
4252,-------------------- The LGFS model is ca...


In [ ]:
all_files_df.iloc[12].str[-1]

title           y
abstract        .
introduction    .
conclusion       
Name: 12, dtype: object

In [ ]:
all_files_df["conclusion"].str[-1] != "."

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: conclusion, Length: 5000, dtype: bool

strip and add dots for the ones without dots.

add dots if the text does not end with it.

In [ ]:
all_files_df["conclusion"] = all_files_df["conclusion"].str.strip()

In [ ]:
all_files_df.loc[(all_files_df["conclusion"] != "") & (all_files_df["conclusion"].str[-1] != "."), "conclusion"] = "."

In [ ]:
all_files_df.to_csv("galactica_output_complete_cleaned.csv")